In [1]:
import os
import gc
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from src import CustomClassifierEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import torch
import time
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.stats import rankdata
from tqdm.auto import tqdm, trange
from sklearn.metrics import accuracy_score
from transformers import get_cosine_schedule_with_warmup,get_linear_schedule_with_warmup, AutoModel, AutoTokenizer, AutoModelForMultipleChoice, AutoConfig
import wandb
pl.seed_everything(56)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

ModuleNotFoundError: No module named 'src'

In [2]:
class CFG:
    class data:
        train_path = 'train.csv'
        test_path = 'test.csv'
        tokenizer = "deepvk/deberta-v1-base"#'microsoft/mdeberta-v3-base'
        num_workers = 8
        nfolds = 5
        batch_size = 32
        use_prefix = False
        max_length = 105 
        seed = 56
    class model:
        model = "deepvk/deberta-v1-base"#'microsoft/mdeberta-v3-base'
        optim = torch.optim.AdamW
        use_only_encoder = False
        grad_acum_steps = 1
        torch_dtype = None
        scheduler= 'cosine'
        warnap_steps = 0.0 #0.25
        num_labels = 50
        label_smoothing = 0.0
        lr = lr_fn = 4e-5
        cls_drop_type = None
        cls_drop = 0.0
        pool = 'mean'
        max_epoches = 10
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        turn_off_drop = True
        num_cycles = 0.5
        eps = 1e-7
        weight_decay = 0.0
        weight_decay_fn = 0.0
        betas = (0.9, 0.999)
        use_lora = False
    seed = 56
    fold_number = 0

def set_wandb_cfg():
    config = {}
    for k,v in CFG.model.__dict__.items():
        if '__' not in k:
            config[k] = v
    for k,v in CFG.data.__dict__.items():
        if '__' not in k:
            config[k] = v
    config['fold_number'] = CFG.fold_number
    return config

In [3]:
def make_df(path,is_test=False):
    data = pd.read_csv(path)
    df = pd.DataFrame()
    if is_test:
        df['label'] = [[0] * 50] * len(df)
    else:
        df['label'] = data.apply(lambda x: [x[f'trend_id_res{i}'] for i in range(50)],axis=1)
    df['text'] = data['text']
    return df

In [4]:
class PLDataset(Dataset):
    def __init__(self, df, tokenizer):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.tokenizer = tokenizer
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]        
        
        encodes = self.tokenizer.encode_plus(
            row['text'],
            max_length=self.cfg.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encodes.input_ids.squeeze(0),
            'attention_mask': encodes.attention_mask.squeeze(0),
            #'token_type_ids': encodes.token_type_ids.squeeze(0),
            'labels': torch.tensor(row['label'])
        }

In [5]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
        
    def prepare_data(self):
        if self.is_prepared: return None
        self.df = make_df(self.cfg.train_path)
        self.test_df = make_df(self.cfg.test_path,is_test=True)
        self.test_df['text'] = self.test_df['text'].fillna('')
        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg.tokenizer)
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup: return None
        kf = MultilabelStratifiedKFold(n_splits=self.cfg.nfolds, shuffle=True, random_state=self.cfg.seed)
        splits = [(x,y) for x,y in  kf.split(self.df.values,np.stack(self.df['label'].values))][CFG.fold_number]
        self.train_df, self.val_df = self.df.iloc[splits[0]], self.df.iloc[splits[1]]
        self.train_dataset = PLDataset(self.train_df,self.tokenizer)
        self.val_dataset = PLDataset(self.val_df,self.tokenizer)
        self.predict_dataset = PLDataset(self.test_df,self.tokenizer)
        self.is_setup = True
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         pin_memory=True,
                         shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)

In [6]:
class AverageMeter():
    def __init__(self):
        self.preds = []
        self.labels = []
        self.history = []
    
    def update(self,y_t,y_p):
        self.labels += y_t
        self.preds += y_p
        
    def clean(self):
        self.preds = []
        self.labels = []

    def calc_metrics(self):
        metrics = {}

        preds = [list(map(lambda y: str(round(y)),x)) for x in self.preds]
        labels = [''.join(map(str,x)) for x in self.labels]
        for i in range(len(self.preds)):
            preds[i][np.argmax(self.preds[i])] = '1'
        preds = [''.join(x) for x in preds]
        
        metrics['accuracy'] = accuracy_score(labels, preds)
        self.history.append(metrics)
        return metrics

In [7]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.model = CustomClassifierEncoder(self.cfg)
        self.avg_meter = AverageMeter()
        self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self, batch):
        output = self.model(**batch)
        return output

    def training_step(self, batch, i):
        logits = self(batch).logits
        loss = self.criterion(logits, batch['labels'].float())
        self.log('train_loss', loss.item())
        return loss
            
    def validation_step(self, batch, i):
        logits = self(batch).logits
        loss = self.criterion(logits, batch['labels'].float())
        self.log('val_loss',loss.item())
        
        preds = logits.sigmoid().tolist()
        labels = batch['labels'].tolist()
        
        self.avg_meter.update(labels,preds)
    
    def predict_step(self, batch, i):
        logits = self(batch).logits
        return logits.sigmoid().tolist()
                
    def on_validation_epoch_end(self):
        metrics = self.avg_meter.calc_metrics()
        self.log_dict(metrics)
        self.avg_meter.clean()
            
    def configure_optimizers(self):        
        optimizer_parameters = [
            {'params': [p for n, p in self.model.model.named_parameters() if not any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': [p for n, p in self.model.model.named_parameters() if any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': 0.0},
            {'params': [p for n, p in self.model.named_parameters() if "model" not in n],
             'lr': self.cfg.lr_fn, 'weight_decay': self.cfg.weight_decay_fn}
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warnap_steps,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warnap_steps)
        else:
            return optim
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [8]:
def train_fold(fold_n=0):
    pl.seed_everything(56)
    CFG.fold_number = fold_n
    
    dm = PLDataModule()
    dm.prepare_data()
    dm.setup(0)
    
    CFG.model.num_training_steps = len(dm.train_dataloader()) * CFG.model.max_epoches
    model = PLModule()
    
    wandb.login(key="31520b01739d418e5d77a11fd8a79a70b189b8bc")
    os.environ['WANDB_API_KEY'] = "31520b01739d418e5d77a11fd8a79a70b189b8bc"
    wandb.init(project='DLS',name='intfloat/multilingual-e5-large-instruct',config=set_wandb_cfg())
    
    lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
    checkpoint_cb = pl.callbacks.ModelCheckpoint(
        dirpath='./outputs/',
        filename='model_{epoch:02d}-{accuracy:.4f}',
        monitor='accuracy',
        mode='max',
        save_last=True
    )

    trainer = pl.Trainer(
        accelerator="gpu",
        precision=32,
        callbacks = [lr_monitor],#[lr_monitor,checkpoint_cb],
        logger = pl.loggers.WandbLogger(save_code=True),
        log_every_n_steps=1,
        accumulate_grad_batches=CFG.model.grad_acum_steps,
        enable_checkpointing=False,
        min_epochs=1,
        devices=1,
        check_val_every_n_epoch=1,
        max_epochs=CFG.model.max_epoches
    )
    trainer.fit(model, datamodule=dm)
    val_preds = trainer.predict(model,dm.val_dataloader())
    test_preds = trainer.predict(model,dm.predict_dataloader())
    hist = model.avg_meter.history[-1]['accuracy']
    model = model.cpu()
    del model, trainer, dm, checkpoint_cb, lr_monitor
    gc.collect()
    torch.cuda.empty_cache()
    return hist,val_preds,test_preds

In [9]:
scores_0, val_preds_0, test_preds_0 = train_fold(0)

Seed set to 56
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: andrewkhl (andlh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                    | Params | Mode 
--------------------------------------------------------------
0 | model     | CustomClassifierEncoder | 124 M  | train
1 | criterion | BCEWithLogitsLoss     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [10]:
scores_1, val_preds_1, test_preds_1 = train_fold(1)

Seed set to 56
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


accuracy,▁▃▅▇▇█████
epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇██
lr-AdamW/pg1,██████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁
lr-AdamW/pg2,████████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
lr-AdamW/pg3,██████▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁
train_loss,█▇▇▆▆▅▅▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▇████
val_loss,█▅▃▂▂▁▁▁▁▁
accuracy,0.49417
epoch,9
lr-AdamW/pg1,0.0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                    | Params | Mode 
--------------------------------------------------------------
0 | model     | CustomClassifierEncoder | 124 M  | train
1 | criterion | BCEWithLogitsLoss       | 0      | train
--------------------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
496.328   Total estimated model params size (MB)
5         Modules in train mode
200       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
def batch_to_device(batch,devide='cuda'):
    for k in batch:
        batch[k] = batch[k].to(device)
    return batch

def train(model, loader, optimizer, epoch, gradient_accumulation_steps, scheduler):
    model.train()
    pbar = tqdm(train_loader)
    for i,batch in enumerate(pbar):
        batch = batch_to_device(batch)

        t = time.time()
        loss = model.training_steb(batch,i)
        t2 = time.time() - t

        pbar.set_description(f"Loss Train: {float(loss.item()) * gradient_accumulation_steps} Time: {t2}")
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    return model

def train_epoch(model, loader, optimizer, scheduler):
    model.train()
    pbar = tqdm(train_loader)
    for i,batch in enumerate(pbar):
        batch = batch_to_device(batch)

        t = time.time()
        loss = model.training_steb(batch,i)
        t2 = time.time() - t

        pbar.set_description(f"Loss Train: {float(loss.item()) * gradient_accumulation_steps} Time: {t2}")
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    return model

@torch.no_grad()
def vallidate_epoch():
    model.eval()
    for i,batch in enumerate(pbar):
        batch = batch_to_device(batch)
        model.validation_steb(batch,i)
    return model.on_validation_epoch_end()

@torch.no_grad()
def predict():
    model.eval()
    for i,batch in enumerate(pbar):
        batch = batch_to_device(batch)
        model.validation_steb(batch,i)
    return model.on_validation_epoch_end()